In [76]:
import requests
import time

from datetime import datetime
from IPython.display import JSON

from unity_sds_client.unity import Unity
from unity_sds_client.unity_services import UnityServices
from unity_sds_client.resources.job_status import JobStatus
from unity_sds_client.unity import UnityEnvironments

In [77]:
unity = Unity(UnityEnvironments.DEV)
unity.set_venue_id("")
process_service = unity.client(UnityServices.PROCESS_SERVICE)

Please enter your Unity username:  drewm
Please enter your Unity password:  ········


In [78]:
process_service.endpoint = "http://k8s-sps-ogcproce-b59e4702b8-1327377322.us-west-2.elb.amazonaws.com:5001"

In [79]:
procs = process_service.get_processes()

In [80]:
for p in procs:
    print(p)

unity_sds_client.resources.Process(
    id="cwl_dag",
    process_version="1.0.0"
    title="Generic CWL Process",
    abstract="This process executes any CWL workflow.",
    keywords="None"
)


In [81]:
process = process_service.get_process("cwl_dag")

In [82]:
print(process)

unity_sds_client.resources.Process(
    id="cwl_dag",
    process_version="1.0.0"
    title="Generic CWL Process",
    abstract="This process executes any CWL workflow.",
    keywords="None"
)


In [73]:
# import pprint
# for key, value in process.inputs.items():
#     print(key)
#     pprint.pprint(value.model_dump())

In [83]:
# Here we submit the job. Currently we submit a dictionary with the required 'inputs', 'outputs','subcscriber' keys if needed
# We need to focus on building out the process_inputs progrmatically.

data = {
  "inputs": {
    "cwl_workflow": "https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/main/demos/echo_message.cwl",
    "cwl_args": "https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/main/demos/echo_message.yaml",
    "request_memory": "8Gi",
    "request_cpu": "4",
    "request_storage": "10Gi"
  },
  "outputs": {
    "result": {
      "transmissionMode": "reference"
    }
  }
}

submitted_job = process.execute(data)

In [84]:
data = {
  "inputs": {
    "cwl_workflow": "https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/preprocess/sbg-preprocess-workflow.cwl",
    "cwl_args": "https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/preprocess/sbg-preprocess-workflow.dev.yml",
    "request_memory": "8Gi",
    "request_cpu": "4",
    "request_storage": "10Gi"
  },
  "outputs": {
    "result": {
      "transmissionMode": "reference"
    }
  }
}

submitted_job = process.execute(data)

In [23]:
import time

status = submitted_job.get_status().status
while status in [JobStatus.ACCEPTED, JobStatus.RUNNING]:
    print("job {} is not complete. Currently in state {} ".format(submitted_job.id, submitted_job.get_status().status))
    time.sleep(5)
    status = submitted_job.get_status().status

print("Job {} completed with status {}".format(submitted_job.id, str(submitted_job.get_status().status)))

AttributeError: 'Execute200ResponseInput' object has no attribute 'get_status'

In [24]:
submitted_job.get_result()

AttributeError: 'Execute200ResponseInput' object has no attribute 'get_result'

In [25]:
status = submitted_job.dismiss()
print(status)

AttributeError: 'Execute200ResponseInput' object has no attribute 'dismiss'

In [55]:
jobs = process_service.get_jobs()

In [56]:
for j in jobs:
    print(j)

unity_sds_client.resources.Job(
    id="52c99990-51f1-4e69-84a0-8e9ff30e3269",
    process="cwl_dag",
    status="accepted",
    inputs=None
)
unity_sds_client.resources.Job(
    id="b525ef73-cac7-415a-8fcc-eabc4a25ffda",
    process="cwl_dag",
    status="accepted",
    inputs=None
)
unity_sds_client.resources.Job(
    id="88b18803-fc3e-4335-a11c-30cfae53bff8",
    process="cwl_dag",
    status="accepted",
    inputs=None
)
